In [1]:
!ls -l /Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets

total 0
drwxrwxr-x@ 34 yaoyuzhu  staff  1088 18 Jun 14:54 shrec_16


In [2]:
cd /Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN

/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN


##导入数据##

In [16]:
# 设置数据目录
DATADIR = "datasets"

# 创建目录并进入
!mkdir -p {DATADIR}

# 下载数据
!wget https://www.dropbox.com/s/w16st84r6wc57u7/shrec_16.tar.gz -P {DATADIR}

# 解压数据并删除压缩包
!tar -xzvf {DATADIR}/shrec_16.tar.gz -C {DATADIR} && rm {DATADIR}/shrec_16.tar.gz

# 提示完成
print(f"✅ Downloaded the data and placed it in: {DATADIR}/shrec_16")


--2025-06-19 09:56:30--  https://www.dropbox.com/s/w16st84r6wc57u7/shrec_16.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 202.160.130.118
Connecting to www.dropbox.com (www.dropbox.com)|202.160.130.118|:443... failed: Operation timed out.
Retrying.

--2025-06-19 09:57:46--  (try: 2)  https://www.dropbox.com/s/w16st84r6wc57u7/shrec_16.tar.gz
Connecting to www.dropbox.com (www.dropbox.com)|202.160.130.118|:443... ^C
tar: Error opening archive: Failed to open 'datasets/shrec_16.tar.gz'
✅ Downloaded the data and placed it in: datasets/shrec_16


##训练模型##

In [19]:
##模型分类准确度
import time
from options.train_options import TrainOptions
from data import DataLoader
from models import create_model
from util.writer import Writer
from main_test import run_test
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "" #禁用cuda

sys.argv = [
    'train.py',
    '--dataroot', 'datasets/shrec_16',
    '--name', 'shrec16',
    '--ncf', '64', '128', '256', '256',
    '--pool_res', '600', '450', '300', '180',
    '--norm', 'group',
    '--resblocks', '1',
    '--flip_edges', '0.2',
    '--slide_verts', '0.2',
    '--num_aug', '20',
    '--niter', '5', #5个epoch
    #'--niter_decay', '100',
    '--niter_decay', '0',
    '--gpu_ids', '-1' #'--gpu_ids', '' 不能传空字符串
]


if __name__ == '__main__':
    opt = TrainOptions().parse()  # 这里不会调用 set_device
    print("Parsing options... gpu_ids:", opt.gpu_ids)
    opt.gpu_ids = []              # 再确保强制用 CPU


    dataset = DataLoader(opt)
    dataset_size = len(dataset)
    print('#training meshes = %d' % dataset_size)

    model = create_model(opt)
    writer = Writer(opt)
    total_steps = 0

    for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
        epoch_start_time = time.time()
        iter_data_time = time.time()
        epoch_iter = 0

        for i, data in enumerate(dataset):
            iter_start_time = time.time()
            if total_steps % opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time
            total_steps += opt.batch_size
            epoch_iter += opt.batch_size
            model.set_input(data)
            model.optimize_parameters()

            if total_steps % opt.print_freq == 0:
                loss = model.loss
                t = (time.time() - iter_start_time) / opt.batch_size
                writer.print_current_losses(epoch, epoch_iter, loss, t, t_data)
                writer.plot_loss(loss, epoch, epoch_iter, dataset_size)

            if i % opt.save_latest_freq == 0:
                print('saving the latest model (epoch %d, total_steps %d)' %
                      (epoch, total_steps))
                model.save_network('latest')

            iter_data_time = time.time()

        if epoch % opt.save_epoch_freq == 0:
            print('saving the model at the end of epoch %d, iters %d' %
                  (epoch, total_steps))
            model.save_network('latest')
            model.save_network(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' %
              (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
        model.update_learning_rate()

        if opt.verbose_plot:
            writer.plot_model_wts(model, epoch)

        if epoch % opt.run_test_freq == 0:
            acc = run_test(epoch)
            writer.plot_acc(acc, epoch)

    writer.close()


------------ Options -------------
arch: mconvnet
batch_size: 16
beta1: 0.9
checkpoints_dir: ./checkpoints
continue_train: False
dataroot: datasets/shrec_16
dataset_mode: classification
epoch_count: 1
export_folder: 
fc_n: 100
flip_edges: 0.2
gpu_ids: []
init_gain: 0.02
init_type: normal
is_train: True
lr: 0.0002
lr_decay_iters: 50
lr_policy: lambda
max_dataset_size: inf
name: shrec16
ncf: [64, 128, 256, 256]
ninput_edges: 750
niter: 5
niter_decay: 0
no_vis: False
norm: group
num_aug: 20
num_groups: 16
num_threads: 3
phase: train
pool_res: [600, 450, 300, 180]
print_freq: 10
resblocks: 1
run_test_freq: 1
save_epoch_freq: 1
save_latest_freq: 250
scale_verts: False
seed: None
serial_batches: False
slide_verts: 0.2
verbose_plot: False
which_epoch: latest
-------------- End ----------------
Parsing options... gpu_ids: []
loaded mean / std from cache
#training meshes = 480
---------- Network initialized -------------
[Network] Total number of parameters : 1.323 M
---------------------------

/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) i

saving the latest model (epoch 1, total_steps 16)
(epoch: 1, iters: 80, time: 0.133, data: 3.935) loss: 3.385 
(epoch: 1, iters: 160, time: 0.159, data: 0.004) loss: 3.392 
(epoch: 1, iters: 240, time: 0.130, data: 0.000) loss: 3.368 
(epoch: 1, iters: 320, time: 0.159, data: 0.007) loss: 3.353 
(epoch: 1, iters: 400, time: 0.141, data: 0.007) loss: 3.401 
(epoch: 1, iters: 480, time: 0.151, data: 0.000) loss: 3.368 
Loading mesh: datasets/shrec_16/bird2/train/T96.obj
Loading mesh: datasets/shrec_16/santa/train/T232.obj
Loading mesh: datasets/shrec_16/cat/train/T217.obj
Loading mesh: datasets/shrec_16/ants/train/T319.obj
Loading mesh: datasets/shrec_16/rabbit/train/T90.obj
Loading mesh: datasets/shrec_16/spiders/train/T244.obj
Loading mesh: datasets/shrec_16/shark/train/T516.obj
Loading mesh: datasets/shrec_16/gorilla/train/T177.obj
Loading mesh: datasets/shrec_16/myScissor/train/T283.obj
Loading mesh: datasets/shrec_16/ants/train/T192.obj
Loading mesh: datasets/shrec_16/armadillo/trai

/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) i

Loading mesh: datasets/shrec_16/bird2/test/T100.obj
Loading mesh: datasets/shrec_16/bird2/test/T494.obj
Loading mesh: datasets/shrec_16/bird2/test/T559.obj
Loading mesh: datasets/shrec_16/bird2/test/T80.obj
Loading mesh: datasets/shrec_16/camel/test/T106.obj
Loading mesh: datasets/shrec_16/camel/test/T431.obj
Loading mesh: datasets/shrec_16/camel/test/T497.obj
Loading mesh: datasets/shrec_16/camel/test/T536.obj
Loading mesh: datasets/shrec_16/cat/test/T123.obj
Loading mesh: datasets/shrec_16/cat/test/T448.obj
Loading mesh: datasets/shrec_16/cat/test/T555.obj
Loading mesh: datasets/shrec_16/cat/test/T74.obj
Loading mesh: datasets/shrec_16/centaur/test/T156.obj
Loading mesh: datasets/shrec_16/centaur/test/T392.obj
Loading mesh: datasets/shrec_16/centaur/test/T565.obj
Loading mesh: datasets/shrec_16/centaur/test/T576.obj
Loading mesh: datasets/shrec_16/horse/test/T117.obj
Loading mesh: datasets/shrec_16/horse/test/T343.obj
Loading mesh: datasets/shrec_16/horse/test/T454.obj
Loading mesh: 

...

Loading mesh: datasets/shrec_16/bird2/test/T100.obj
Loading mesh: datasets/shrec_16/bird2/test/T494.obj
Loading mesh: datasets/shrec_16/bird2/test/T559.obj
Loading mesh: datasets/shrec_16/bird2/test/T80.obj
Loading mesh: datasets/shrec_16/camel/test/T106.obj
Loading mesh: datasets/shrec_16/camel/test/T431.obj
Loading mesh: datasets/shrec_16/camel/test/T497.obj
Loading mesh: datasets/shrec_16/camel/test/T536.obj
Loading mesh: datasets/shrec_16/cat/test/T123.obj
Loading mesh: datasets/shrec_16/cat/test/T448.obj
Loading mesh: datasets/shrec_16/cat/test/T555.obj
Loading mesh: datasets/shrec_16/cat/test/T74.obj
Loading mesh: datasets/shrec_16/centaur/test/T156.obj
Loading mesh: datasets/shrec_16/centaur/test/T392.obj
Loading mesh: datasets/shrec_16/centaur/test/T565.obj
Loading mesh: datasets/shrec_16/centaur/test/T576.obj
Loading mesh: datasets/shrec_16/horse/test/T117.obj
Loading mesh: datasets/shrec_16/horse/test/T343.obj
Loading mesh: datasets/shrec_16/horse/test/T454.obj
Loading mesh: 

/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) i

saving the latest model (epoch 5, total_steps 1936)
(epoch: 5, iters: 80, time: 0.178, data: 4.304) loss: 3.152 
(epoch: 5, iters: 160, time: 0.158, data: 0.000) loss: 3.020 
(epoch: 5, iters: 240, time: 0.158, data: 0.009) loss: 2.970 
(epoch: 5, iters: 320, time: 0.158, data: 0.005) loss: 2.833 
(epoch: 5, iters: 400, time: 0.207, data: 0.000) loss: 2.949 
(epoch: 5, iters: 480, time: 0.203, data: 0.011) loss: 2.905 
Loading mesh: datasets/shrec_16/woman/train/T332.obj
Loading mesh: datasets/shrec_16/ants/train/T509.obj
Loading mesh: datasets/shrec_16/centaur/train/T95.obj
Loading mesh: datasets/shrec_16/cat/train/T488.obj
Loading mesh: datasets/shrec_16/gorilla/train/T26.obj
Loading mesh: datasets/shrec_16/armadillo/train/T279.obj
Loading mesh: datasets/shrec_16/dino_ske/train/T190.obj
Loading mesh: datasets/shrec_16/bird2/train/T500.obj
Loading mesh: datasets/shrec_16/snake/train/T181.obj
Loading mesh: datasets/shrec_16/myScissor/train/T283.obj
Loading mesh: datasets/shrec_16/plier

/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) i

Loading mesh: datasets/shrec_16/alien/test/T124.obj
Loading mesh: datasets/shrec_16/alien/test/T411.obj
Loading mesh: datasets/shrec_16/alien/test/T511.obj
Loading mesh: datasets/shrec_16/alien/test/T547.obj
Loading mesh: datasets/shrec_16/ants/test/T1.obj
Loading mesh: datasets/shrec_16/ants/test/T347.obj
Loading mesh: datasets/shrec_16/ants/test/T484.obj
Loading mesh: datasets/shrec_16/ants/test/T518.obj
Loading mesh: datasets/shrec_16/armadillo/test/T138.obj
Loading mesh: datasets/shrec_16/armadillo/test/T473.obj
Loading mesh: datasets/shrec_16/armadillo/test/T55.obj
Loading mesh: datasets/shrec_16/armadillo/test/T574.obj
Loading mesh: datasets/shrec_16/bird1/test/T118.obj
Loading mesh: datasets/shrec_16/bird1/test/T41.obj
Loading mesh: datasets/shrec_16/bird1/test/T589.obj
Loading mesh: datasets/shrec_16/bird1/test/T66.obj
Loading mesh: datasets/shrec_16/flamingo/test/T131.obj
Loading mesh: datasets/shrec_16/flamingo/test/T389.obj
Loading mesh: datasets/shrec_16/flamingo/test/T578.

##Tensorboard看训练过程##

In [5]:
!tensorboard --logdir=/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/runs


Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6007/ (Press CTRL+C to quit)
^C


In [20]:
import os
import numpy as np

root_dir = '/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN'

def check_ve_in_npz(npz_path):
    try:
        data = np.load(npz_path)
        has_ve = 've' in data.files
        return has_ve
    except Exception as e:
        print(f"Error loading {npz_path}: {e}")
        return False

npz_files_with_ve = []
npz_files_without_ve = []

for dirpath, _, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.npz'):
            full_path = os.path.join(dirpath, filename)
            if check_ve_in_npz(full_path):
                npz_files_with_ve.append(full_path)
            else:
                npz_files_without_ve.append(full_path)

print(f"包含've'字段的 .npz 文件 ({len(npz_files_with_ve)} 个):")
for f in npz_files_with_ve:
    print(f)

print(f"\n不包含've'字段的 .npz 文件 ({len(npz_files_without_ve)} 个):")
for f in npz_files_without_ve:
    print(f)


包含've'字段的 .npz 文件 (9244 个):
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T471_008.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T471_009.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T478_019.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T471_019.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T471_018.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T105_009.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T4_001.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T4_015.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T4_000.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T105_008.npz
/Users/yaoyuzhu/MeshCNN/MeshCNN/MeshCNN/datasets/shrec_16/gorilla/test/cache/T

##导出测试结果##

In [33]:
import sys
from options.test_options import TestOptions
from data import DataLoader
from models import create_model
from util.writer import Writer

def run_test(epoch=-1):
    print('Running Test')

    sys.argv = [
        'main_test.py',
        '--dataroot', 'datasets/shrec_16',
        '--name', 'shrec16',
        '--export_folder', 'meshes',
        '--gpu_ids', '-1',
        '--norm', 'group',
        '--ncf', '64', '128', '256', '256',
        '--pool_res', '600', '450', '300', '180',
        '--fc_n', '100', 
        '--resblocks', '1'
    ]

    opt = TestOptions().parse()
    opt.serial_batches = True  # no shuffle
    dataset = DataLoader(opt)
    model = create_model(opt)
    writer = Writer(opt)

    writer.reset_counter()
    for i, data in enumerate(dataset):
        model.set_input(data)
        ncorrect, nexamples = model.test()
        writer.update_counter(ncorrect, nexamples)
    writer.print_acc(epoch, writer.acc)
    return writer.acc

if __name__ == '__main__':
    run_test()


Running Test
loaded mean / std from cache
loading the model from ./checkpoints/shrec16/latest_net.pth


/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
/Users/yaoyuzhu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


Loading mesh: datasets/shrec_16/alien/test/T124.obj
Loading mesh: datasets/shrec_16/alien/test/T411.obj
Loading mesh: datasets/shrec_16/alien/test/T511.obj
Loading mesh: datasets/shrec_16/alien/test/T547.obj
Loading mesh: datasets/shrec_16/ants/test/T1.obj
Loading mesh: datasets/shrec_16/ants/test/T347.obj
Loading mesh: datasets/shrec_16/ants/test/T484.obj
Loading mesh: datasets/shrec_16/ants/test/T518.obj
Loading mesh: datasets/shrec_16/armadillo/test/T138.obj
Loading mesh: datasets/shrec_16/armadillo/test/T473.obj
Loading mesh: datasets/shrec_16/armadillo/test/T55.obj
Loading mesh: datasets/shrec_16/armadillo/test/T574.obj
Loading mesh: datasets/shrec_16/bird1/test/T118.obj
Loading mesh: datasets/shrec_16/bird1/test/T41.obj
Loading mesh: datasets/shrec_16/bird1/test/T589.obj
Loading mesh: datasets/shrec_16/bird1/test/T66.obj
Loading mesh: datasets/shrec_16/flamingo/test/T131.obj
Loading mesh: datasets/shrec_16/flamingo/test/T389.obj
Loading mesh: datasets/shrec_16/flamingo/test/T578.

In [41]:
import sys
print(sys.executable)



/Users/yaoyuzhu/opt/anaconda3/bin/python


In [42]:
!which python

/Users/yaoyuzhu/miniconda3/bin/python


In [43]:
import sys
!{sys.executable} -m pip install trimesh pyglet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.0/712.0 kB 918.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 2.3 MB/s eta 0:00:0000:0100:01
